# Kaggle - LLM Science Exam
> Use LLMs to answer difficult science questions

<img src="https://www.kaggle.com/competitions/54662/images/header">

# 🎯 | Motivation

* In this notebook, we will demonstrate the usage of the multi-backend capabilities of `KerasCore` and `KerasNLP` for the **MultipleChoice** infernece.

# 📌 | Updates
* `v04` - DebertaV3Base, 2 folds
* `v03` - DistilBertBase, 2 folds, fixed ShuffleOption
* `v02` - DistilBertBase, 2 folds ShuffleOption

# 🛠 | Install Libraries 

In [1]:
!pip install /kaggle/input/llm-science-exam-lib-ds/keras_core-0.1.4-py3-none-any.whl --no-deps
!pip install /kaggle/input/llm-science-exam-lib-ds/keras_nlp-0.6.1-py3-none-any.whl --no-deps

Processing /kaggle/input/llm-science-exam-lib-ds/keras_core-0.1.4-py3-none-any.whl
Processing /kaggle/input/llm-science-exam-lib-ds/keras_nlp-0.6.1-py3-none-any.whl


# 📚 | Import Libraries 

In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras_core as keras 
import keras_core.backend as K


import jax
import tensorflow as tf
# from tensorflow import keras
# import tensorflow.keras.backend as K

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

from glob import glob
from tqdm.notebook import tqdm
import gc

Using JAX backend.


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

## Library Version

In [3]:
print("TensorFlow:", tf.__version__)
# print("JAX:", jax.__version__)
print("Keras:", keras.__version__)
print("KerasNLP:", keras_nlp.__version__)

TensorFlow: 2.12.0
Keras: 0.1.4
KerasNLP: 0.6.1


# ⚙️ | Configuration

In [4]:
class CFG:
    verbose = 0  # Verbosity
    device = 'TPU'  # Device
    seed = 42  # Random seed
    batch_size = 4  # Batch size
    drop_remainder = True  # Drop incomplete batches
    ckpt_dir = "/kaggle/input/llm-science-exam-kerascore-kerasnlp-tpu-ds"  # Name of pretrained models
    sequence_length = 200  # Input sequence length
    class_names = list("ABCDE")  # Class names [A, B, C, D, E]
    num_classes = len(class_names)  # Number of classes
    class_labels = list(range(num_classes))  # Class labels [0, 1, 2, 3, 4]
    label2name = dict(zip(class_labels, class_names))  # Label to class name mapping
    name2label = {v: k for k, v in label2name.items()}  # Class name to label mapping

# ♻️ | Reproducibility 
Sets value for random seed to produce similar result in each run.

In [5]:
keras.utils.set_random_seed(CFG.seed)

# 💾 | Hardware
Following codes automatically detects hardware (TPU or GPU). 

In [6]:
def get_device():
    "Detect and intializes GPU/TPU automatically"
    try:
        # Connect to TPU
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() 
        # Set TPU strategy
        strategy = tf.distribute.TPUStrategy(tpu)
        print(f'> Running on TPU', tpu.master(), end=' | ')
        print('Num of TPUs: ', strategy.num_replicas_in_sync)
        device=CFG.device
    except:
        # If TPU is not available, detect GPUs
        gpus = tf.config.list_logical_devices('GPU')
        ngpu = len(gpus)
         # Check number of GPUs
        if ngpu:
            # Set GPU strategy
            strategy = tf.distribute.MirroredStrategy(gpus) # single-GPU or multi-GPU
            # Print GPU details
            print("> Running on GPU", end=' | ')
            print("Num of GPUs: ", ngpu)
            device='GPU'
        else:
            # If no GPUs are available, use CPU
            print("> Running on CPU")
            strategy = tf.distribute.get_strategy()
            device='CPU'
    return strategy, device

In [7]:
# Initialize GPU/TPU/TPU-VM
strategy, CFG.device = get_device()
CFG.replicas = strategy.num_replicas_in_sync

> Running on GPU | Num of GPUs:  1


# 📁 | Dataset Path 

In [8]:
BASE_PATH = '/kaggle/input/kaggle-llm-science-exam'

# 📖 | Meta Data 
* **train.csv** - a set of 200 questions with the answer column. Each question consists of a `prompt` (the question), 5 options labeled `A`, `B`, `C`, `D`, and `E`, and the correct answer labeled `answer` (this holds the label of the most correct answer, as defined by the generating LLM).
* **test.csv** - similar to train.csv except it doesn't have `answer` column.  It has ~4000 questions that may be different is subject matter.
* **sample_submission.csv** - is the test sample submission.
    * `id`: number id of the question
    * `prediction`: top 3 labels for your prediction. Once a correct label has been scored for an individual question in the test set, that label is no longer considered relevant for that question, and additional predictions of that label are skipped in the calculation

## Test Data

In [9]:
test_df = pd.read_csv(f'{BASE_PATH}/test.csv')  # Read CSV file into a DataFrame

# Display information about the train data
print("# Test Data: {:,}".format(len(test_df)))
print("# Sample:")
display(test_df.head(2))

# Test Data: 200
# Sample:


,id,prompt,A,B,C,D,E
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...


## Contextualize Options

Our approach entails furnishing the model with question and answer pairs, as opposed to employing a single question for all five options. In practice, this signifies that for the five options, we will supply the model with the same set of five questions combined with each respective answer choice (e.g., `(Q + A)`, `(Q + B)`, and so on). This analogy draws parallels to the practice of revisiting a question multiple times during an exam to promote a deeper understanding of the problem at hand.

In [10]:
# Define a function to create options based on the prompt and choices
def make_options(row):
    row['options'] = [f"{row.prompt}\n{row.A}",  # Option A
                      f"{row.prompt}\n{row.B}",  # Option B
                      f"{row.prompt}\n{row.C}",  # Option C
                      f"{row.prompt}\n{row.D}",  # Option D
                      f"{row.prompt}\n{row.E}"]  # Option E
    return row


In [11]:
test_df = test_df.apply(make_options, axis=1)  # Apply the make_options function to each row in df
test_df.head(2)  # Display the first 2 rows of df

,id,prompt,A,B,C,D,E,options
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,[Which of the following statements accurately ...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,[Which of the following is an accurate definit...


# 🍽️ | Preprocessing

**What it does:** The preprocessor takes input strings and transforms them into a dictionary (`token_ids`, `padding_mask`) containing preprocessed tensors. This process starts with tokenization, where input strings are converted into sequences of token IDs.

**Why it's important:** Initially, raw text data is complex and challenging for modeling due to its high dimensionality. By converting text into a compact set of tokens, such as transforming `"The quick brown fox"` into `["the", "qu", "##ick", "br", "##own", "fox"]`, we simplify the data. Many models rely on special tokens and additional tensors to understand input. These tokens help divide input and identify padding, among other tasks. Making all sequences the same length through padding boosts computational efficiency, making subsequent steps smoother.

Explore the following pages to access the available preprocessing and tokenizer layers in **KerasNLP**:
- [Preprocessing](https://keras.io/api/keras_nlp/preprocessing_layers/)
- [Tokenizers](https://keras.io/api/keras_nlp/tokenizers/)

In [12]:
vocab_path = '/kaggle/input/keras-nlp-deberta-v3-base-en-vocab-ds/vocab.spm'
tokenizer= keras_nlp.models.DebertaV3Tokenizer(vocab_path)
preprocessor= keras_nlp.models.DebertaV3Preprocessor(tokenizer, sequence_length=CFG.sequence_length)

Now, let's examine what the output shape of the preprocessing layer looks like. The output shape of the layer can be represented as $(num\_choices, sequence\_length)$.

In [13]:
outs = preprocessor(test_df.options.iloc[0])  # Process options for the first row

# Display the shape of each processed output
for k, v in outs.items():
    print(k, ":", v.shape)

token_ids : (5, 200)
padding_mask : (5, 200)


We'll use the `preprocessing_fn` function to transform each text option using the `dataset.map(preprocessing_fn)` method.

In [14]:
def preprocess_fn(text, label=None):
    text = preprocessor(text)  # Preprocess text
    return (text, label) if label is not None else text  # Return processed text and label if available

# 🍚 | DataLoader

The code below sets up a robust data flow pipeline using `tf.data.Dataset` for data processing. Notable aspects of `tf.data` include its ability to simplify pipeline construction and represent components in sequences.

To learn more about `tf.data`, refer to this [documentation](https://www.tensorflow.org/guide/data).

In [15]:
def build_dataset(texts, labels=None, batch_size=32,
                  cache=False, drop_remainder=True,
                  augment=False, repeat=False, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNE option
    slices = (texts,) if labels is None else (texts, keras.utils.to_categorical(labels, num_classes=5))  # Create slices
    ds = tf.data.Dataset.from_tensor_slices(slices)  # Create dataset from slices
    ds = ds.cache() if cache else ds  # Cache dataset if enabled
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  # Map preprocessing function
    ds = ds.repeat() if repeat else ds  # Repeat dataset if enabled
    opt = tf.data.Options()  # Create dataset options
    if shuffle: 
        ds = ds.shuffle(shuffle, seed=CFG.seed)  # Shuffle dataset if enabled
        opt.experimental_deterministic = False
    ds = ds.with_options(opt)  # Set dataset options
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)  # Batch dataset
    ds = ds.prefetch(AUTO)  # Prefetch next batch
    return ds  # Return the built dataset

## Fetch Train/test Dataset

The function below generates the training and testation datasets for a given fold.

In [16]:
def get_test_dataset(test_df):
    test_texts = test_df.options.tolist()  # Extract testation texts
    
    # Build testation dataset
    test_ds = build_dataset(test_texts, labels=None,
                             batch_size=min(CFG.batch_size*CFG.replicas, len(test_df)), cache=False,
                             shuffle=False, drop_remainder=False, repeat=False)
    
    return test_ds  # Return datasets and dataframes

# 🤖 | Modeling



## Classifier for Multiple-Choice Tasks

When dealing with multiple-choice questions, instead of giving the model the question and all options together `(Q + A + B + C ...)`, we provide the model with one option at a time along with the question. For instance, `(Q + A)`, `(Q + B)`, and so on. Once we have the prediction scores (logits) for all options, we combine them using the `Softmax` function to get the ultimate result. If we had given all options at once to the model, the text's length would increase, making it harder for the model to handle. The picture below illustrates this idea:

![Model Diagram](https://pbs.twimg.com/media/F3NUju_a8AAS8Fq?format=png&name=large)

<div align="center"><b> Picture Credict: </b> <a href="https://twitter.com/johnowhitaker"> @johnowhitaker </a> </div></div><br>

From a coding perspective, remember that we use the same model for all five options, with shared weights. Despite the figure suggesting five separate models, they are, in fact, one model with shared weights. Another point to consider is the the input shapes of Classifier and MultipleChoice.

* Input shape for **Multiple Choice**: $(batch\_size, num\_choices, seq\_length)$
* Input shape for **Classifier**: $(batch\_size, seq\_length)$

Certainly, it's clear that we can't directly give the data for the multiple-choice task to the model because the input shapes don't match. To handle this, we'll use **slicing**. This means we'll separate the features of each option, like $feature_{(Q + A)}$ and $feature_{(Q + B)}$, and give them one by one to the NLP classifier. After we get the prediction scores $logits_{(Q + A)}$ and $logits_{(Q + B)}$ for all the options, we'll use the Softmax function, like $\operatorname{Softmax}([logits_{(Q + A)}, logits_{(Q + B)}])$, to combine them. This final step helps us make the ultimate decision or choice.

> Note that in the classifier, we set `num_classes=1` instead of `5`. This is because the classifier produces a single output for each option. When dealing with five options, these individual outputs are joined together and then processed through a softmax function to generate the final result, which has a dimension of `5`.

In [17]:
# Selects one option from five
class SelectOption(keras.layers.Layer):
    def __init__(self, index, **kwargs):
        super().__init__(**kwargs)
        self.index = index

    def call(self, inputs):
        # Selects a specific slice from the inputs tensor
        return inputs[:, self.index, :]
    
    def get_config(self):
        # For serialize the model
        base_config = super().get_config()
        config = {
            "index": self.index,
        }
        return {**base_config, **config}

def build_model():
    # Define input layers
    inputs = {
        "token_ids": keras.Input(shape=(5, None), dtype=tf.int32, name="token_ids"),
        "padding_mask": keras.Input(shape=(5, None), dtype=tf.int32, name="padding_mask"),
    }
    # Create a DebertaV3Classifier model
    classifier = keras_nlp.models.DebertaV3Classifier.from_preset(
        CFG.preset,
        preprocessor=None,
        num_classes=1 # one output per one option, for five options total 5 outputs
    )
    logits = []
    # Loop through each option (Q+A), (Q+B) etc and compute associted logits
    for option_idx in range(5):
        option = {k: SelectOption(option_idx, name=f"{k}_{option_idx}")(v) for k, v in inputs.items()}
        logit = classifier(option)
        logits.append(logit)
        
    # Compute final output
    logits = keras.layers.Concatenate(axis=-1)(logits)
    outputs = keras.layers.Softmax(axis=-1)(logits)
    model = keras.Model(inputs, outputs)
    return model

## Ckpt processing
For some reason, `keras.models.load_model` requires write access as `/kaggle/input` doesn't have that access it throws error. Workaround is to simply copy the `ckpts` to other directory then load the model.

In [18]:
# Get the checkpoint directory and name
ckpt_dir = CFG.ckpt_dir
ckpt_name = ckpt_dir.split('/')[3]

# Copy the checkpoints to a new directory in the /kaggle directory
!cp -r {ckpt_dir} /kaggle/{ckpt_name}

# List all the checkpoint paths in the new directory
new_ckpt_dir = f"/kaggle/{ckpt_name}"
ckpt_paths = glob(os.path.join(new_ckpt_dir, '*.keras'))

print("Total CKPT:", len(ckpt_paths))

Total CKPT: 2


# 🧪 | Prediction

## Inference

In [19]:
# Initialize an array to store predictions for each fold
fold_preds = np.zeros(shape=(len(test_df), 5), dtype='float32')

# Iterate through each checkpoint path
for ckpt_path in tqdm(ckpt_paths):
    # Load the pre-trained model from the checkpoint
    model = keras.models.load_model(
        ckpt_path,
        compile=False,
        custom_objects={"SelectOption": SelectOption}  # Use the custom layer
    )
    
    # Get the test dataset
    test_ds = get_test_dataset(test_df)
    
    # Generate predictions using the model
    preds = model.predict(
        test_ds,
        batch_size=min(CFG.batch_size * CFG.replicas * 2, len(test_df)),  # Set batch size
        verbose=1
    )
    
    # Add predictions to fold_preds and average over checkpoints
    fold_preds += preds / len(ckpt_paths)
    
    # Clean up by deleting the model and collecting garbage
    del model
    gc.collect()

  0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/keras_core/src/saving/serialization_lib.py:686: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/opt/conda/lib/python3.10/site-packages/keras_core/src/backend/jax/numpy.py:103: UserWarning: Explicitly requested dtype int64 requested in arange is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return jnp.arange(start, stop, step=step, dtype=dtype)
/opt/conda/lib/python3.10/site-packages/keras_core/src/backend/jax/core.py:40: UserWarning: Explicitly requested dt

50/50 ━━━━━━━━━━━━━━━━━━━━ 82s 954ms/step
50/50 ━━━━━━━━━━━━━━━━━━━━ 82s 978ms/step


## Check Prediction

In [20]:
# Format predictions and true answers
pred_answers = np.array(list('ABCDE'))[np.argsort(-fold_preds)]

# Check 5 Predictions
print("# Predictions\n")
for i in range(2):
    row = test_df.iloc[i]
    question  = row.prompt
    pred_answer = pred_answers[i, 0]
    print(f"❓ Question {i+1}:\n{question}\n")
    print(f"🤖 Predicted Answer: {pred_answer}\n{row[pred_answer]}\n")
    print("-"*90, "\n")

# Predictions

❓ Question 1:
Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?

🤖 Predicted Answer: B
MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.

------------------------------------------------------------------------------------------ 

❓ Question 2:
Which of the following is an accurate definition of dynamic scaling in self-similar systems?

🤖 Predicted Answer: D
Dynamic scaling refers to the non-evolution of self-similar systems, where data obtained from snapshots at fixed times is dissimilar to the respective data taken from snapshots of any earlier or later time. This dissimilarity is tested by a certain time-independent stochastic variable y.

----------------------------------------------------------

# 📮 | Submission

In [21]:
# Create a DataFrame to store the submission
sub_df = test_df[["id"]].copy()

# Get the top 3 predicted answers
top3_answers = pred_answers[:, :3]

# Format the predicted answers as strings
format_pred = [' '.join(x) for x in top3_answers]

# Add the formatted predictions to the submission DataFrame
sub_df["prediction"] = format_pred

# Save Submission
sub_df.to_csv('submission.csv',index=False)

# Display the first 2 rows of the submission DataFrame
sub_df.head(2)

,id,prediction
0,0,B D C
1,1,D C A
